#### Custom operators based on SQL functions

A variant to the use case for `Operators.op.is_comparison` is when we __aren't using an operator, but a SQL function__. The typical example of this use case is the _PostgreSQL PostGIS functions_; however, __any SQL function on any database that resolves to a binary condition may apply__. To suit this use case, the `FunctionElement.as_comparison()` method can __modify any SQL function__, such as those invoked from the _func namespace_, to indicate to the ORM that the function produces a comparison of two expressions. The below example illustrates this with the _Geoalchemy2_ library.

In [1]:
from geoalchemy2 import Geometry
from sqlalchemy import Column, Integer, func
from sqlalchemy.orm import declarative_base, foreign, relationship

In [2]:
Base = declarative_base()

In [3]:
class Polygon(Base):
    __tablename__ = "polygon"
    
    id = Column(Integer, primary_key=True)
    geom = Column(Geometry("POLYGON", srid=4326))
    points = relationship(
        "Point", viewonly=True,
        primaryjoin="func.ST_Contains(foreign(Polygon.geom), Point.geom).as_comparison(1, 2)",
    )

In [4]:
class Point(Base):
    __tablename__ = "point"
    
    id = Column(Integer, primary_key=True)
    geom = Column(Geometry("POINT", srid=4326))

Above, the `FunctionElement.as_comparison()` indicates that the `func.ST_Contains()` SQL function is __comparing__ the _Polygon.geom_ and _Point.geom_ expressions. The `foreign()` annotation additionally notes which column takes on the _"foreign key" role_ in this particular relationship.